In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [2]:
train_dir = "../data/mri/train"
val_dir = "../data/mri/val"

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=5,
    zoom_range=0.05
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode="binary"
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode="binary"
)

train_generator.class_indices

Found 346 images belonging to 2 classes.
Found 346 images belonging to 2 classes.


{'NORMAL': 0, 'STONE': 1}

In [4]:
base_model = ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)

output = Dense(1, activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=output)

In [5]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [6]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)

Epoch 1/10
22/22 [==============================] - 26s 1s/step - loss: 0.7485 - accuracy: 0.5058 - val_loss: 0.7912 - val_accuracy: 0.4769
Epoch 2/10
22/22 [==============================] - 23s 1s/step - loss: 0.7907 - accuracy: 0.4971 - val_loss: 0.7376 - val_accuracy: 0.5231
Epoch 3/10
22/22 [==============================] - 23s 1s/step - loss: 0.7395 - accuracy: 0.5405 - val_loss: 0.8820 - val_accuracy: 0.4769
Epoch 4/10
22/22 [==============================] - 23s 1s/step - loss: 0.7344 - accuracy: 0.5289 - val_loss: 0.6801 - val_accuracy: 0.5665
Epoch 5/10
22/22 [==============================] - 23s 1s/step - loss: 0.8233 - accuracy: 0.5116 - val_loss: 0.6690 - val_accuracy: 0.5838
Epoch 6/10
22/22 [==============================] - 23s 1s/step - loss: 0.7615 - accuracy: 0.5347 - val_loss: 0.7021 - val_accuracy: 0.4740
Epoch 7/10
22/22 [==============================] - 23s 1s/step - loss: 0.6707 - accuracy: 0.5867 - val_loss: 0.6418 - val_accuracy: 0.6329
Epoch 8/10
22/22 [==

In [7]:
model.save("../model/mri_kidney_stone_model.h5")

C:\Users\ASUS\OneDrive\Desktop\MedLens\venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
